In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("creditcard.csv")

In [ ]:
data

In [ ]:
sample = data.sample(frac=0.2, random_state=0)

In [ ]:
data.shape

In [ ]:
sample.shape

In [ ]:
fraud = data[data['Class']==1]
normal = data[data['Class']==0]

In [ ]:
outlier = len(fraud)/len(normal)
outlier

In [ ]:
data['Class'].value_counts().plot(kind= 'bar')

In [ ]:
sns.distplot(data['Class'])

In [ ]:
data['Class'].value_counts().plot(kind='bar')
plt.title('Distinction b/w fraud and normal')
plt.xlabel('class-fraud(2)/normal(0)')
plt.ylabel('number of transactions')f

In [ ]:
fraud.Amount.describe()

In [ ]:
data.info()

In [ ]:
matrix = data.corr()
top_feature = matrix.index
plt.figure(figsize=[30,30])
sns.heatmap(data[top_feature].corr(), annot=True, cbar=False, cmap='RdYlGn')


In [ ]:
independenFeature = [column for column in data.columns if column != 'Class']

In [ ]:
dependentFeature = [column for column in data.columns if column == 'Class']

In [ ]:
independenFeature

In [ ]:
dependentFeature

In [ ]:
X = sample[independenFeature]
y =sample[dependentFeature]

In [ ]:
X = sample.drop(['Class'], axis = 1)
y = sample['Class']

In [ ]:
classifiers = dict(IsolationForest = IsolationForest(n_estimators=100, max_samples=len(X), 
                                       contamination=outlier,random_state=0, verbose=0),
                 LocalOutlierFactor = LocalOutlierFactor(n_neighbors=20, algorithm='auto', 
                                              leaf_size=30, metric='minkowski',
                                              p=2, metric_params=None, contamination=outlier),
                 SupportVectorMachine = OneClassSVM(kernel='rbf', degree=3, gamma=0.1,nu=0.05, 
                                         max_iter=-1, random_state=0)
                  )

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
n_outliers = len(fraud)
for i, (clf_name,clf) in enumerate(classifiers.items()):
    #Fit the data and tag outliers
    if clf_name == "LocalOutlierFactor":
        y_pred = clf.fit_predict(X)
        scores_prediction = clf.negative_outlier_factor_
    elif clf_name == "SupportVectorMachine":
        clf.fit(X)
        y_pred = clf.predict(X)
    else:    
        clf.fit(X)
        scores_prediction = clf.decision_function(X)
        y_pred = clf.predict(X)
    #Reshape the prediction values to 0 for Valid transactions , 1 for Fraud transactions
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    n_errors = (y_pred != y).sum()
    # Run Classification Metrics
    print("{}: {}".format(clf_name,n_errors))
    print("Accuracy Score :")
    print(accuracy_score(y,y_pred))
    print("Classification Report :")
    print(classification_report(y,y_pred))